<h1 style="color: teal">Lecture 4 - Cython</h1>

<strong style="color: #1B2A49">E. Margarita Palacios Vargas<br>
Fundación Universitaria Konrad Lorenz</strong>

---

<h2 style="color: teal">Cython</h2>

Cython is a Python compiler that makes writing C extensions for Python as easy as Python itself. Cython translates Python code to C/C++ code but additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code.

This makes Cython the ideal language for wrapping external C libraries, and for fast C modules that speed up the execution of Python code.

<h4 style="color: teal">Installation</h4>

<p>You can use either <code>pip</code> or <code>Anaconda</code> to install Cython:</p>

<div style="display: flex; text-align: center;">
  <div style="flex: 1; margin-right: 50px;">
    <pre><code class="language-bash">pip install cython</code></pre>
  </div>

  <div style="flex: 1; margin-right: 50px;">
    <p>or</p>
  </div>

  <div style="flex: 1; margin-right: 50px;">
    <pre><code class="language-bash">conda install cython</code></pre>
  </div>
</div>

<h4 style="color: teal">Usage</h4>

In Jupyter notebook, you can load the Cython extension as follows:

In [2]:
%load_ext Cython

Then, prefix a cell with the `%%cython` marker to compile it

In [ ]:
%%cython
# The quiet is because Windows is annoying with Cython

cdef long long a = 0 # Better than old syntax -> a: cython.int = 0
cdef long long i
for i in range(int(1e7)):
    a += i
print(a)

Using the `--annotate` option will show Cython’s code analysis:

In [ ]:
%%cython --annotate

cdef long long a = 0
cdef long long i

for i in range(int(1e7)):
    a += i
print(a)

Of course, we can test performance of compiled and native Python:

In [ ]:
%%cython

def sum_numbers_cython(int n):
    # This code includes type declarations (e.g., cdef int) for variables,
    # which allows Cython to generate more efficient C code.
    cdef int result = 0
    cdef int i
    for i in range(1, n + 1):
        result += i
    return result

In [ ]:
def sum_numbers(n):
    result = 0
    for i in range(1, n+1):
        result += i
    return result

In [ ]:
N = 2000
%timeit sum_numbers(N)
%timeit sum_numbers_cython(N)

---

<h2 style="color: teal">Main differences between Cython and Python code</h2>

| **Feature**              | **Python**                 | **Cython Equivalent**                       |
|---------------------------|----------------------------|---------------------------------------------|
| Variables                 | `x = 0`                   | `cdef int x = 0`                            |
| Functions                 | `def f(x): ...`           | `cdef int f(int x): ...` <br> `cpdef` for Python + C access |
| Loops                     | `for i in range(10**7):`  | `cdef int i` <br> `for i in range(10**7):` |
| Arrays (NumPy)            | `np.zeros((1000,))`       | `cdef double[:] arr = np.zeros(1000)`       |
| External C functions      | *Not possible directly*   | `cdef extern from "math.h": double sin(double x)` |
| Compiler directives       | *None*                    | `# cython: boundscheck=False, wraparound=False` |


---

<h2 style="color: teal">Cythonize outside Jupyter</h2>

`cythonize` is a function provided by the `Cython` library that is used to compile `Cython` code (which presents itself as `.pyx` files) into `C` code and then into `Python` extension modules. In short, it converts `Cython` code into a form that can be imported in `Python`, usually with much better performance for computationally intensive tasks. `cythonize` simplifies the build process for developers who want to optimize `Python` using `Cython`.

Suppose we have the following structure inside the `examples/` folder:

```
examples/
├── module1.pyx
├── setup.py
└── main.py
```

### 1. The `.pyx` file

`module1.pyx` contains the Cython code we want to compile:

```cython
# examples/module1.pyx
def square(int x):
    return x * x
```

### 2. The `setup.py` build script

The `setup.py` file uses `cythonize` to compile the `.pyx` code:

```python
# examples/setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("module1.pyx")
)
```

### 3. Build the extension

From the `examples/` folder, run:

```bash
cd examples
python setup.py build_ext --inplace
```

This command compiles your Cython code into a shared library:

- On Linux/macOS: `module1.cpython-311-x86_64-linux-gnu.so`
- On Windows: `module1.cp311-win_amd64.pyd`

The `--inplace` flag ensures the compiled file is created directly in the `examples/` directory so it can be imported right away. You can also trigger this programmatically inside Python (e.g. in a notebook):

In [ ]:
from os import system
system("cd examples/ && python setup.py build_ext --inplace")

### 4. Use the compiled module

Finally, in `main.py` you can import and use your Cython function like a normal Python module:

```python
# examples/main.py
from module1 import function

result = function(10) # Output: 20
print(result)

```

Run it from the shell:

```shell
python examples/main.py
```

---

<h2 style="color: teal">Functions and libraries</h2>

<h4 style="color: teal">1. Numpy</h4>

Consider the following piece of code, where we define a function that calculates the aritmethic mean of a given array.

In [ ]:
%%cython

# To define a function we use the "cpdef" instruction
cpdef double cython_mean(long long[:] array):
    cdef int n = array.shape[0]
    cdef double sum_val = 0.
    cdef int ii
    for ii in range(n):
        sum_val += array[ii]

    return sum_val / n

and now we test this function with numpy arrays:

In [ ]:
import numpy as np

arr = np.random.randint(1, 100, int(1e8), dtype = np.int64)
result = cython_mean(arr)
print(result)

In [ ]:
# Let us test times
%timeit cython_mean(arr)
%timeit np.mean(arr)

Now consider the following code, which aims to perform matrix multiplication using `Cython`:

In [3]:
%%cython

import numpy as np
cimport numpy as np

cpdef np.ndarray[double, ndim = 2] matrix_multiply(np.ndarray[double, ndim = 2] A, np.ndarray[double, ndim = 2] B):
    cdef int m, n, p
    m = A.shape[0]
    n = A.shape[1]
    p = B.shape[1]
    
    cdef np.ndarray[double, ndim = 2] result = np.zeros((m, p), dtype = np.float64)
    
    cdef int i, j, k
    for i in range(m):
        for j in range(p):
            for k in range(n):
                result[i, j] += A[i, k] * B[k, j]
    
    return result

In [4]:
# Create two NumPy arrays
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)

# Call the Cython function for matrix multiplication
result = matrix_multiply(A, B)

# Print the result
print(result)

[[246.54908125 246.93046803 254.73557631 ... 252.21689701 244.72653593
  253.61978807]
 [240.68926004 242.48004361 248.16050777 ... 247.57737147 229.78032075
  239.52326159]
 [246.63367537 249.5989984  252.32224549 ... 250.95098232 242.88089472
  250.81961474]
 ...
 [248.45193162 247.26246521 265.4301939  ... 253.60962911 247.98755238
  258.59690621]
 [252.83634714 257.03004697 257.37644692 ... 257.77276936 251.39779244
  257.38407631]
 [243.78471068 248.72711063 257.16611872 ... 243.55620294 244.61846491
  250.11790667]]


In [5]:
%timeit matrix_multiply(A, B)
%timeit A.dot(B)

11.2 s ± 327 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
36.2 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<h4 style="color: teal">2. Cython</h4>

Consider the following `Cython` code, used to calculate the CDF of a normal distribution:

In [6]:
%%cython
cimport numpy as np
from scipy.stats import norm

cpdef np.ndarray[double] cdf_normal(np.ndarray[double] x, double loc, double scale):
    return norm.cdf(x, loc, scale)

In [7]:
import numpy as np
from scipy.stats import norm

# Generate test data
x = np.random.normal(0, 1, int(1e7))

# Calculate CDF using SciPy
%timeit norm.cdf(x, loc = 0, scale = 1)

# Calculate CDF using the Cython-optimized function
%timeit cdf_normal(x, loc = 0, scale = 1)

1.11 s ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.12 s ± 4.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<h4 style="color: teal">3. Sympy</h4>

Let's do a bit of symbolic differentiation:

In [8]:
%%cython

from sympy import symbols, diff

cpdef symbolic_differentiation():
    x = symbols('x')
    expression = x**3 + 2*x**2 + 3*x + 4
    derivative = diff(expression, x)
    return derivative

In [9]:
x = symbols('x')
expression = x**3 + 2*x**2 + 3*x + 4

%timeit diff(expression, x)
%timeit symbolic_differentiation()

172 μs ± 3.37 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
228 μs ± 2.53 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


---

<h2 style="color: teal">Regarding Cython's speed</h2>


It is important to remember that **Cython does not automatically make code faster**. Cython is most effective when:

- You can add **static type declarations** (`cdef int x`)  
- You can **avoid Python-level overhead** (e.g., object boxing/unboxing, bounds checking)  
- You are working on **tight loops or custom algorithms** not already optimized in libraries  

---

<h4 style="color: teal">Why NumPy is often faster than our Cython code?</h4>

- **NumPy functions (`dot`, `sum`, etc.) call BLAS/LAPACK**  
  These are highly optimized C/Fortran libraries (MKL, OpenBLAS, Accelerate) for Linear Algebra stuff, with:
  - Cache blocking
  - SIMD vectorization (Single Instruction, Multiple Data)
  - Multi-threading


- **Naive Cython loops are single-threaded**  
  Even with typed variables, a plain `for` loop:
  - Does no caché optimization
  - Executes in pure C, but with no vectorization
  - Still pays some overhead if using `np.ndarray[...]` indexing

---

<h4 style="color: teal">Rule of Thumb</h4>

- Use **NumPy / SciPy** first → they are already written in C/Fortran and optimized for decades.  
- Use **Cython** when:
  - You need to accelerate **pure Python code** that has no good NumPy equivalent  
  - You want to write **custom low-level algorithms** (e.g., specialized simulations, parsing, tight loops)  
  - You need to interface with **external C/C++ libraries**

---

<h4 style="color: teal">Takeaway</h4>

If you compare Cython loops to NumPy’s `dot` or `sum`, NumPy will usually win.  
Cython shines when you step **outside the NumPy toolbox** or need **custom logic** that BLAS/LAPACK can’t cover.
